# Ejemplo de patrones: Reunión (join)

En este ejercicio vamos a realizar la reunión (o join) de dos datasets. En primer lugar tenemos el fichero de posts con el que hemos trabajado anteriormente, y por otro lado tenemos otro fichero que contiene información de los usuarios. Sobre el fichero de los usuarios, tiene una serie de campos separados por tabuladores. Los campos son los siguientes: 

"user_ptr_id", "reputation", "gold", "silver", "bronze" 

El primer campo almacena un identificador para el usuario, mientras que los siguientes almacenan una serie de valores que sirven para calificar al usuario en función de sus contribuciones en el foro.

El resultado de la ejecución de este programa MapReduce debe ser que, para cada post del foro, se muestren los siguientes datos: 

"id", "title", "tagnames", "author_id", "added_at", "score", "user_ptr_id", "reputation", "gold", "silver", "bronze" 

De estos datos, los campos "id", "title", "tagnames", "author_id", "added_at", y  "score" vienen del fichero que contiene los posts y el resto de los campos vienen del fichero de los usuarios.  

Para una posible implementación es de interés incluir en la salida del map un nuevo campo sintético, que sirva para ordenar la información de un usuario y los posts que ha publicado. Este campo puede tomar el valor “A” para la información de usuarios y “B” para la información de los posts. De esta forma, los reducers recibirán, para cada usuario, primero su información de usuario y seguidamente sus posts.  

In [5]:
! mkdir -p ejemplo-patrones/join

In [6]:
import os
os.chdir("/media/notebooks/ejemplo-patrones/join")

In [7]:
! pwd

/media/notebooks/ejemplo-patrones/join


In [8]:
%%writefile mapper.py
#!/usr/bin/env python

import sys
import csv

 
reader = csv.reader(sys.stdin, delimiter='\t')
writer = csv.writer(sys.stdout, delimiter='\t', quotechar='"', quoting=csv.QUOTE_ALL)

# PON TU CÓDIGO AQUÍ

Overwriting mapper.py


In [9]:
%%writefile reducer.py
#!/usr/bin/env python
# Los valores que contengan como source A son los datos de usuarios, y B datos de posts

import sys
import csv


reader = csv.reader(sys.stdin, delimiter='\t')
writer = csv.writer(sys.stdout, delimiter='\t', quotechar='"', quoting=csv.QUOTE_ALL)

currentUser = None

id = title = tagnames = author_id = parent_id = abs_parent_id = added_at = score = reputation = gold = silver = bronze = ""
source = None
# PON TU CÓDIGO AQUÍ

Overwriting reducer.py


In [10]:
! hdfs dfs -ls /tmp/forumdata

Found 3 items
-rw-r--r--   3 root supergroup       1774 2019-08-12 08:03 /tmp/forumdata/forum1.tsv
-rw-r--r--   3 root supergroup  120313812 2019-08-12 08:03 /tmp/forumdata/forum_node.tsv
-rw-r--r--   3 root supergroup     530989 2019-08-12 08:03 /tmp/forumdata/forum_users.tsv


In [11]:
! hdfs dfs -rm /tmp/salida-join/*
! hdfs dfs -rmdir /tmp/salida-join

rm: `/tmp/salida-join/*': No such file or directory
rmdir: `/tmp/salida-join': No such file or directory


In [12]:
! hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /tmp/forumdata/forum_node.tsv -input /tmp/forumdata/forum_users.tsv \
-output /tmp/salida-join

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob5400627880740110649.jar tmpDir=null
19/08/12 10:04:42 INFO client.RMProxy: Connecting to ResourceManager at yarnmaster/172.21.0.3:8032
19/08/12 10:04:42 INFO client.RMProxy: Connecting to ResourceManager at yarnmaster/172.21.0.3:8032
19/08/12 10:04:42 INFO mapred.FileInputFormat: Total input paths to process : 2
19/08/12 10:04:42 INFO mapreduce.JobSubmitter: number of splits:3
19/08/12 10:04:43 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1565590814059_0019
19/08/12 10:04:43 INFO impl.YarnClientImpl: Submitted application application_1565590814059_0019
19/08/12 10:04:43 INFO mapreduce.Job: The url to track the job: http://yarnmaster:8088/proxy/application_1565590814059_0019/
19/08/12 10:04:43 INFO mapreduce.Job: Running job: job_1565590814059_0019
19/08/12 10:04:48 INFO mapreduce.Job: Job job_1565590814059_0019 running in uber mode : false
19/08/12 10:04:48 INFO mapreduce.Job

In [13]:
! hdfs dfs -tail /tmp/salida-join/part-00000

"	"0"	"4"
"12003576"	""	"cs258 "	"100119321"	"2012-08-16 21:54:08.588876+00"	"1"	"76"	"0"	"0"	"4"
"12003561"	"You should add an item to the returned list each time enqueue() or dequeue() is called."	"cs258 spoiler final-2 meta"	"100119321"	"2012-08-16 04:36:19.794722+00"	"0"	"76"	"0"	"0"	"4"
"12003489"	""	"cs258 "	"100119376"	"2012-08-10 22:22:06.93941+00"	"0"	"1"	"0"	"0"	"1"
"12003513"	"type of testing"	"cs258 unit1-9 m-48720273"	"100119386"	"2012-08-12 05:14:14.80794+00"	"1"	"1"	"0"	"0"	"1"
"12003589"	""	"cs258 "	"100119409"	"2012-08-19 22:21:12.14117+00"	"0"	"41"	"0"	"0"	"2"
"12003569"	""	"cs258 "	"100119415"	"2012-08-16 15:03:46.57102+00"	"0"	"41"	"0"	"0"	"2"
"12003565"	""	"cs258 "	"100119430"	"2012-08-16 13:08:25.856929+00"	"0"	"71"	"0"	"0"	"3"
"12003557"	""	"cs258 "	"100119430"	"2012-08-15 15:47:49.224421+00"	"0"	"71"	"0"	"0"	"3"
"12003566"	""	"cs258 "	"100119430"	"2012-08-16 13:10:39.810898+00"	"2"	"71"	"0"	"0"	"3"
"12003564"	""	"cs258 "	"100119430"	"2012-08-16 12:47:39.764575+0